<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/EDA_%EA%B3%BC%EC%A0%95_0_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 21.2 MB/s eta 0:00:00


In [2]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [3]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

In [8]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

# 데이터불러오기

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   NSMC_0.02_증분대상100개.csv   NSMC_0.1_증분대상_500개.csv   가족


In [29]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/NSMC_0.02_증분대상100개.csv', encoding='utf-8-sig')

In [30]:
df.head(3)

,Unnamed: 0.1,Unnamed: 0,id,document,label
0,5000,152610,9621933,소재는 좋았지만 10년전임을 감안해도 촌스러운 bgm개연성없는 전개박중훈 연기변신 ...,0
1,5001,165761,9977101,몰입하기가 너무 힘들었다.. 그냥 그림이나 줄거리로 볼때 상당히 아름다운 영화같았는...,0
2,5002,154012,7517589,개막장 드라마&작가의 끝을 보여주네요..오늘 보다가 도저히 못참고 채널돌렸음.어찌 ...,0


In [31]:
# 증강 적용
df['text_swap_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

In [32]:
# 증강 적용
df['text_swap_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_10'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_10'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_10'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_10'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_11'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_11'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_11'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_11'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_12'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_12'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_12'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_12'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))



In [ ]:
# 증강 적용
df['text_swap_13'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_13'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_13'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_13'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_14'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_14'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_14'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_14'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_15'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_15'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_15'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_15'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_16'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_16'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_16'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_16'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_17'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_17'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_17'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_17'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_18'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_18'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_18'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_18'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_19'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_19'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_19'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_19'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_20'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_20'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_20'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_20'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))



In [ ]:
# 증강 적용
df['text_swap_21'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_21'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_21'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_21'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_22'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_22'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_22'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_22'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_23'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_23'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_23'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_23'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_24'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_24'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_24'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_24'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_25'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_25'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_25'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_25'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_26'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_26'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_26'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_26'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_27'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_27'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_27'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_27'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_28'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_28'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_28'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_28'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_29'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_29'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_29'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_29'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))




In [ ]:
df['text_swap_30'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_30'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_30'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_30'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_31'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_31'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_31'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_31'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_32'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_32'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_32'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_32'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_33'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_33'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_33'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_33'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_34'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_34'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_34'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_34'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_35'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_35'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_35'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_35'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_36'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_36'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_36'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_36'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_37'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_37'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_37'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_37'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_38'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_38'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_38'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_38'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_39'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_39'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_39'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_39'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))



In [ ]:
df['text_swap_40'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_40'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_40'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_40'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_41'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_41'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_41'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_41'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_42'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_42'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_42'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_42'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_43'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_43'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_43'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_43'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_44'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_44'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_44'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_44'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))

df['text_swap_45'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=5))
df['text_insert_45'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=5))
df['text_delete_45'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_45'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=5))

df['text_swap_46'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=5))
df['text_insert_46'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=5))
df['text_delete_46'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_46'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=5))

df['text_swap_47'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=5))
df['text_insert_47'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=5))
df['text_delete_47'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_47'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=5))

df['text_swap_48'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=5))
df['text_insert_48'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=5))
df['text_delete_48'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_48'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=5))

df['text_swap_49'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=5))
df['text_insert_49'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_49'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.1))
df['text_replace_49'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=5))



In [ ]:
df.to_csv('NSMC_0.02_aug_eda_final.csv', encoding='utf-8-sig')